In [184]:
import pandas as pd
import numpy as np
from clases.NeuronalNetwork import SimpleNeuralNetwork
from clases.Optimizer import SGDOptimizer, AdamOptimizer
from clases.Trainer import Trainer
from clases.LossFunction import *
from clases.ActivationFunction import *

import warnings
warnings.filterwarnings("ignore")

## MNIST Data

### Load Data

In [185]:
data = pd.read_csv('../datos/mnist_data.csv')
data = np.array(data)

### Split Data

In [186]:
m, n = data.shape
np.random.shuffle(data)

test_size = int(0.1 * m)
data_test = data[:test_size, :].T  # El 10% inicial para el conjunto de prueba

# El conjunto provisional restante (90%)
remaining_data = data[test_size:, :]

train_size = int(0.8 * m)
data_train = remaining_data[:train_size, :].T  # El siguiente 80% para entrenamiento
data_validation = remaining_data[train_size:, :].T  # El resto 10% para validación

X_train, Y_train = data_train[1:], data_train[0].astype(int)
X_val, Y_val = data_validation[1:], data_validation[0].astype(int)
X_test, Y_test = data_test[1:], data_test[0].astype(int)

### Train Data

#### SGD Optimizer

In [187]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = SGDOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=130)

--------------------------------------------------------------------
| Epoch   0 | time:  0.18s | val loss 2.280 | valid accuracy 0.160 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.12s | val loss 1.877 | valid accuracy 0.369 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.13s | val loss 1.318 | valid accuracy 0.630 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.13s | val loss 0.972 | valid accuracy 0.725 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.13s | val loss 0.797 | valid accuracy 0.767 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.14s | val loss 0.696 | valid accuracy 0.792 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.12s | val loss 0.628 | valid accuracy 0.806 |
----------------------------------

#### Adam Optimizer

In [188]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = AdamOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=130)

--------------------------------------------------------------------
| Epoch   0 | time:  0.18s | val loss 2.214 | valid accuracy 0.205 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.13s | val loss 1.262 | valid accuracy 0.563 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.12s | val loss 0.763 | valid accuracy 0.762 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.12s | val loss 0.523 | valid accuracy 0.846 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.12s | val loss 0.425 | valid accuracy 0.869 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.12s | val loss 0.375 | valid accuracy 0.885 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.14s | val loss 0.349 | valid accuracy 0.898 |
----------------------------------

## Iris Data

### Load the Data

In [189]:
data = pd.read_csv('../datos/Iris.csv')
data = np.array(data)

### Label Encoder

In [190]:
target_column = data[:, -1]

unique_classes, encoded_labels = np.unique(target_column, return_inverse=True)

data[:, -1] = encoded_labels

### Standard Scaler

In [191]:
features = data[:, :-1].astype(np.float64)

means = features.mean(axis=0)
stds = features.std(axis=0)

scaled_features = (features - means) / stds

data[:, :-1] = scaled_features

### Split Data

In [192]:
m, n = data.shape
np.random.shuffle(data)

test_size = int(0.1 * m)
data_test = data[:test_size, :].T  # El 10% inicial para el conjunto de prueba

# El conjunto provisional restante (90%)
remaining_data = data[test_size:, :]

train_size = int(0.8 * m)
data_train = remaining_data[:train_size, :].T  # El siguiente 80% para entrenamiento
data_validation = remaining_data[train_size:, :].T  # El resto 10% para validación

X_train, Y_train = data_train[:-1].astype(np.float64), data_train[-1].astype(int)
X_val, Y_val = data_validation[:-1].astype(np.float64), data_validation[-1].astype(int)
X_test, Y_test = data_test[:-1].astype(np.float64), data_test[-1].astype(int)

### Train Data

#### SGDOptimizer

In [193]:
model = SimpleNeuralNetwork(input_size=4, output_size=3, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = SGDOptimizer(learning_rate=1)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=130)

--------------------------------------------------------------------
| Epoch   0 | time:  0.00s | val loss 1.100 | valid accuracy 0.600 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.00s | val loss 0.986 | valid accuracy 0.600 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.00s | val loss 0.348 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.00s | val loss 0.193 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.00s | val loss 0.128 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.00s | val loss 0.091 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.00s | val loss 0.069 | valid accuracy 1.000 |
----------------------------------

#### AdamOptimizer

In [194]:
model = SimpleNeuralNetwork(input_size=4, output_size=3, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = AdamOptimizer(learning_rate=0.1)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=130)

--------------------------------------------------------------------
| Epoch   0 | time:  0.00s | val loss 1.096 | valid accuracy 0.600 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.00s | val loss 0.287 | valid accuracy 0.933 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.00s | val loss 0.057 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.00s | val loss 0.011 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.00s | val loss 0.008 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.00s | val loss 0.006 | valid accuracy 1.000 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.00s | val loss 0.006 | valid accuracy 1.000 |
----------------------------------